In [ ]:
import imaplib
import email
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re
from selenium.webdriver.support.ui import Select

# # === Step 1: Set up Selenium WebDriver ===
# driver = webdriver.Chrome()

# Define Functions

def login(driver, username, password):
    driver.get("https://cms.gift.edu.in/index.php?r=site%2Flogin")
    username_element = driver.find_element(By.ID, "loginform-username")
    password_element = driver.find_element(By.ID, "loginform-password")
    username_element.send_keys(username)
    password_element.send_keys(password)

    send_otp_button = driver.find_element(By.NAME, "login-button")
    send_otp_button.click()
    print("Login credentials entered and OTP requested.")

def get_otp_from_email(email_user, email_pass):
    try:
        imap_host = 'imap.gmail.com'
        mail = imaplib.IMAP4_SSL(imap_host)
        mail.login(email_user, email_pass)

        time.sleep(10)  # Wait for OTP email to arrive
        mail.select('inbox')

        status, email_ids = mail.search(None, '(FROM "cms@gift.edu.in")')
        email_ids = email_ids[0].split()

        if len(email_ids) == 0:
            print("No emails found from the sender.")
            return None

        latest_email_id = email_ids[-1]
        status, data = mail.fetch(latest_email_id, '(RFC822)')
        raw_email = data[0][1]
        msg = email.message_from_bytes(raw_email)

        email_body = None
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain" and "attachment" not in str(part.get("Content-Disposition")):
                    email_body = part.get_payload(decode=True).decode()
                    break
        else:
            email_body = msg.get_payload(decode=True).decode()

        if email_body is None:
            print("Failed to extract the email body.")
            return None

        otp = extract_otp_from_email(email_body)
        return otp

    except Exception as e:
        print(f"Error occurred while retrieving OTP: {e}")
        return None

    finally:
        if 'mail' in locals():
            mail.logout()

def extract_otp_from_email(email_body):
    otp_match = re.search(r'\b\d{4}\b', email_body)
    if otp_match:
        return otp_match.group(0)
    return None

def enter_otp(driver, otp):
    otp_element = driver.find_element(By.ID, "otptextbox")
    otp_element.send_keys(otp)
    validate_otp_button = driver.find_element(By.XPATH, "//button[contains(@class, 'btn-success') and contains(text(), 'Validate OTP')]")
    validate_otp_button.click()
    print("OTP entered and validated.")

def navigate_to_online_exam(driver):
    try:
        online_exam_link = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Online"))
        )
        online_exam_link.click()
        print("Navigated to 'Online Examination' link.")
    except TimeoutException:
        print("Failed to find the 'Online Examination' link.")

def access_online_examination(driver):
    try:
        access_link = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='edusec-link-box-text']/a[contains(@href, 'onlineexam%2Fexam-system%2Findex')]"))
        )
        access_link.click()
        print("Accessed 'Online Examination'.")
    except TimeoutException:
        print("Failed to find the 'Access Online Examination' link.")

def add_new_question(driver):
    try:
        dropdown_menu = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Question Bank')]"))
        )
        dropdown_menu.click()
        print("Opened 'Question Bank' dropdown.")

        add_new_link = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//ul[@class='dropdown-menu']//a[contains(text(), 'Add new')]"))
        )
        add_new_link.click()
        print("Clicked on 'Add new' link.")
    except TimeoutException:
        print("Failed to click on 'Add new' link.")

def click_next_button(driver):
    try:
        next_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Next')]"))
        )
        next_button.click()
        print("Clicked on 'Next' button.")
    except TimeoutException:
        print("Failed to find the 'Next' button.")

def write_to_tinymce_editor_question(driver, iframe_xpath, text):
    try:
        tinymce_iframe = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, iframe_xpath))
        )
        driver.switch_to.frame(tinymce_iframe)
        print("Switched to TinyMCE iframe.")

        editor_body = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, "tinymce"))
        )
        editor_body.clear()
        editor_body.send_keys(text)
        print(f"Entered text in TinyMCE editor: {text}")

        driver.switch_to.default_content()
    except TimeoutException:
        print("Failed to locate the TinyMCE editor or iframe.")

def write_to_tinymce_editor_options(iframe_xpath, option_text, option_number):
    try:
        # Step 1: Wait for a few seconds to ensure the iframe has loaded (or use an explicit wait)
        time.sleep(0)  # Optionally, adjust the wait time depending on how long it takes for the iframe to load

        # Step 2: Switch to the iframe containing the TinyMCE editor for the option
        tinymce_iframe = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, iframe_xpath))
        )
        driver.switch_to.frame(tinymce_iframe)
        print(f"Switched to TinyMCE iframe for Option {option_number}.")

        # Step 3: Locate the <body> tag inside the TinyMCE editor and input the text
        editor_body = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, "tinymce"))
        )
        editor_body.clear()  # Optional: Clear any existing text
        editor_body.send_keys(option_text)
        print(f"Successfully entered text in TinyMCE editor for Option {option_number}.")

        # Step 4: Switch back to the default content after writing
        driver.switch_to.default_content()
        print(f"Switched back to the main content after Option {option_number}.")

    except TimeoutException:
        print(f"Failed to locate the TinyMCE editor or iframe for Option {option_number}.")
    except NoSuchElementException:
        print(f"No such element found for TinyMCE iframe or editor body for Option {option_number}.")


def select_correct_option(option_value):
    try:
        # Locate the radio button based on the value provided
        option_radio = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f"//input[@type='radio' and @name='score' and @value='{option_value}']"))
        )
        # Click the radio button
        option_radio.click()
        print(f"Successfully selected Option {int(option_value) + 1} as the correct answer.")

    except TimeoutException:
        print(f"Failed to locate the radio button for Option {int(option_value) + 1}.")
    except NoSuchElementException:
        print(f"No such element found for Option {int(option_value) + 1} radio button.")


def select_category(driver, category_name):
    try:
        # Step 1: Locate the <select> dropdown element
        category_dropdown = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.NAME, "cid"))
        )
        
        # Step 2: Create a Select object
        select = Select(category_dropdown)

        # Step 3: Select the option with visible text provided in category_name
        select.select_by_visible_text(category_name)
        print(f"Successfully selected the '{category_name}' category.")

    except TimeoutException:
        print("Failed to locate the dropdown for categories.")
    except NoSuchElementException:
        print(f"No such option found in the dropdown: '{category_name}'.")



def select_difficulty_level(driver, difficulty_level="Easy"):
    try:
        # Step 1: Locate the <select> dropdown element for difficulty level
        difficulty_dropdown = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.NAME, "lid"))
        )
        
        # Step 2: Create a Select object
        select = Select(difficulty_dropdown)

        # Step 3: Select the option with visible text provided in difficulty_level
        select.select_by_visible_text(difficulty_level)
        print(f"Successfully selected the '{difficulty_level}' difficulty level.")

    except TimeoutException:
        print("Failed to locate the dropdown for difficulty levels.")
    except NoSuchElementException:
        print(f"No such option found in the dropdown: '{difficulty_level}'.")



def submit_question(driver):
    try:
        # Wait for the "Submit" button to be clickable and click it using XPath with text
        submit_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='Submit']"))
        )
        submit_button.click()
        print("Successfully clicked on the 'Submit' button.")
    except TimeoutException:
        print("Failed to find the 'Submit' button.")
    except NoSuchElementException:
        print("No such element found for the 'Submit' button.")




In [ ]:
###################################### For Uploding Quiz Questions #####################################

import json

if __name__ == "__main__":
    # Step 1: Set up Selenium WebDriver
    driver = webdriver.Chrome()

    try:
        # Login to the system, 
        login(driver, "Enter Gift User Name", "Enter Gift password") ##########################################

        # Retrieve and validate OTP
        otp = get_otp_from_email("Enter you gift mail id", "Enter you gift mail password") ###############################
        if otp:
            enter_otp(driver, otp)
            print("OTP validated successfully.")

            # Navigate to Online Examination
            navigate_to_online_exam(driver)
            access_online_examination(driver)

            # Load questions from JSON file
            with open('write_module_json_file_name', 'r') as f: ##########################################
                questions = json.load(f)

            # Add and submit questions
            for question in questions:
                add_new_question(driver)
                click_next_button(driver)
                write_to_tinymce_editor_question(driver, "//iframe[contains(@id, 'question_ifr')]", question["question_text"])

                # Write each option
                option_iframes = [
                    "//iframe[contains(@id, 'option')]",
                    "//iframe[contains(@id, 'mce_3')]",
                    "//iframe[contains(@id, 'mce_4')]",
                    "//iframe[contains(@id, 'mce_5')]"
                ]
                for i, option in enumerate(question["options"]):
                    write_to_tinymce_editor_options(option_iframes[i], option, f"Option {i + 1}")

                # Set correct option, category, and difficulty
                select_correct_option(question["correct_option"] - 1)
                select_category(driver, "AI")
                select_difficulty_level(driver, question["difficulty"])
                time.sleep(0.5)
                # Submit the question
                submit_question(driver)

        else:
            print("Failed to retrieve OTP.")
    
    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        pass
        # Close the browser
        # driver.quit()


In [ ]:
####################################### For Only Login ###########################################
import json

if __name__ == "__main__":
    # Step 1: Set up Selenium WebDriver
    driver = webdriver.Chrome()

    try:
        # Login to the system
        login(driver, "Enter_your_Gift_User_name", "Enter_your_Gift_password")

        # Retrieve and validate OTP
        otp = get_otp_from_email("Enter_your_Gift_mail_id", "Enter_you_git_mail_password")
        if otp:
            enter_otp(driver, otp)
            print("OTP validated successfully.")
        else:
            print("Failed to retrieve OTP.")
    
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        pass
        # Close the browser
        # driver.quit()
